# RF

In [ ]:
import os
import shap
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

folder_path = 'J:\\'

file_list = glob.glob(folder_path + '*.xlsx')

r2_scores = []
rmse_scores = []
mae_scores = []
sows = []
matures = []

df_y_pre = pd.DataFrame()

df_accurate = pd.DataFrame()

for file_name in file_list:
    print(file_name)
    sow = file_name.split('-')[3] 
    mature = file_name.split('-')[4]
    data = pd.read_excel(file_name)

    X = data.iloc[:, 5:]
    y = data['Yield'].values
    
    imputer = SimpleImputer()
    X = imputer.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf = RandomForestRegressor(n_estimators=100,max_depth=10,random_state=42)

    scores = cross_val_score(rf, X_train, y_train, cv=10, scoring='r2')

    rf.fit(X_train, y_train)

    y_pred = rf.predict(X)

    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    mae = mean_absolute_error(y, y_pred)

    print('R2: {:.4f}'.format(r2))
    print('RMSE: {:.4f}'.format(rmse))
    print('MAE: {:.4f}'.format(mae))

    sows.append(sow)
    matures.append(mature)
    r2_scores.append(r2)
    rmse_scores.append(rmse)
    mae_scores.append(mae)

    col_name = str(sow)+'-'+str(mature)
    df_y_pre[col_name] = y_pred

df_accurate["r2_scores"] = r2_scores
df_accurate["rmse_scores"] = rmse_scores
df_accurate["mae_scores"] = mae_scores
df_accurate["sows"] = sows
df_accurate["gsl"] = matures

df_y_pre.to_excel("J:\.xlsx",index = False)
df_accurate.to_excel("J:\.xlsx",index = False)

# XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import glob
import pandas as pd
import numpy as np
import xgboost as xgb
import shap

folder_path = "J:\\"

file_list = glob.glob(folder_path + '*.xls')

r2_scores = []
rmse_scores = []
mae_scores = []
sows = []
matures = []

SHAP = pd.DataFrame()

df_y_pre = pd.DataFrame()

df_accurate = pd.DataFrame()


for file_name in file_list:
    print(file_name)
    sow = file_name.split('-')[1] 
    mature = file_name.split('-')[2]
    col_name = str(sow)+'-'+str(mature)
    
    data = pd.read_excel(file_name)
    num_columns = len(data.columns)
    
    X = data.iloc[:, :].values
    y = data['Yield'].values
    
    imputer = SimpleImputer()
    X = imputer.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    
    xgb_model = xgb.XGBRegressor(n_estimators=100,max_depth=10,subsample=0.8, random_state=42)
    
    scores = cross_val_score(xgb_model, X_train, y_train, cv=10, scoring='r2')

    xgb_model.fit(X_train, y_train)

    y_pred = xgb_model.predict(X)

    sows.append(sow)
    matures.append(mature)
    r2_scores.append(r2)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    
    col_name = str(sow)+'-'+str(mature)
    df_y_pre[col_name] = y_pred


df_accurate["r2_scores"] = r2_scores
df_accurate["rmse_scores"] = rmse_scores
df_accurate["mae_scores"] = mae_scores
df_accurate["sows"] = sows
df_accurate["gsl"] = matures

df_y_pre.to_excel("J:\.xlsx",index = False)
df_accurate.to_excel("J:\.xlsx",index = False)

# LSTM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from keras.layers import Dropout
import tensorflow as tf
from sklearn.impute import SimpleImputer
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.compat.v1.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import Dense

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

folder_path = "J:\\"

file_list = glob.glob(folder_path + '*.xlsx')

def cross_val_generator(X, y, k):
    fold_size = len(X) // k
    for i in range(k):
        test_idx = range(i*fold_size, (i+1)*fold_size)
        train_idx = list(set(range(len(X))) - set(test_idx))
        yield X[train_idx], y[train_idx], X[test_idx], y[test_idx]

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

r2_scores = []
rmse_scores = []
mae_scores = []
sows = []
matures = []

SHAP = pd.DataFrame()

df_y_pre = pd.DataFrame()

df_accurate = pd.DataFrame()

for file_name in file_list:
    print(file_name)
    sow = file_name.split('-')[3] 
    mature = file_name.split('-')[4]
    col_name = str(sow)+'-'+str(mature)
    
    data = pd.read_excel(file_name)
    data = data[(data['year'] >= 2017) & (data['year'] <= 2020)]
    
    data.set_index(['year','ID'], inplace=True)

    X = data.iloc[:, :].values
    y = data['yield'].values
    df_y_pre['yield'] = y
    imputer = SimpleImputer()
    X = imputer.fit_transform(X)
    
    sc = MinMaxScaler()
    X = sc.fit_transform(X)
    y = sc.fit_transform(y.reshape(-1, 1))
    
    lstm = Sequential()
    lstm.add(LSTM(units=64, return_sequences=True, input_shape=(X.shape[1], 1)))
    lstm.add(Dropout(0.1))
    lstm.add(LSTM(units=64, return_sequences=True))#, return_sequences=True
    lstm.add(Dropout(0.1))
    lstm.add(LSTM(units=64))
    lstm.add(Dropout(0.1))
    lstm.add(Dense(units=1))
    lstm.compile(optimizer='adam', loss='mean_squared_error')#'adam'

    batch_size = 64
    epochs = 200
    k = 10  # cross-validation

    start_time = time.time()
    for i, (X_train, y_train, X_test, y_test) in enumerate(cross_val_generator(X, y, k)):
        # print(f'Fold {i+1}:')
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        lstm.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)  
    lstm.save("J:\"+col_name+"-lstm.h5")
    
    end_time = time.time()
    elapsed_time_model1 = end_time - start_time
    print("Time span：",elapsed_time_model1)
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    y_pred = lstm.predict(X)
    y_pred = sc.inverse_transform(y_pred)
    y_true = data['yield'].values
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse_score = np.sqrt(mae)
    r2 = r2_score(y_true, y_pred)
    print('RMSE:',rmse_score)
    print('MAE:', mae)
    print('R2:', r2)

    sows.append(sow)
    matures.append(mature)
    r2_scores.append(r2)
    rmse_scores.append(rmse_score)
    mae_scores.append(mae)
    
    df_y_pre[col_name] = list(y_pred)
    
df_accurate["r2_scores"] = r2_scores
df_accurate["rmse_scores"] = rmse_scores
df_accurate["mae_scores"] = mae_scores
df_accurate["sows"] = sows
df_accurate["gsl"] = matures

df_y_pre.to_excel("J:\\",index = False)
df_accurate.to_excel("J:\\",index = False)